
# Análisis del Perfil Molecular de *Staphylococcus aureus* en Estado Plantónico y de *Biofilm* Empleando Espectrometría de Masas MALDI-TOF y Herramientas de Machine Learning 
---
Willy Colan Avendaño


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

### Biofil Vs Planctónico

#### Código de las Extracciones: Ácido Fórmico Acetonitrilo (AF-ACN), Cloroformo - metanol ($HCCl_3$-$MetOH$) y buffer de lisis (BL)

In [ ]:
import glob
fn = []
all_MS = pd.DataFrame()
#nombres=['MF']
for f in glob.glob("C:/Users/Usuario/OneDrive - UNIVERSIDAD INDUSTRIAL DE SANTANDER/Documents/Analisis_Datos_Quimica-main/Datos plan B/29213-43300/Extracción 1/*.txt"):
    df = pd.read_csv(f, header=None, delimiter=' ' )
    all_MS = pd.concat([all_MS, df], axis=1)
    fn.append(f) # Guardamos parte del nombre del archivo como una etiqueta de cada muestra
Datamz = all_MS[0] # realciones m/z
Data = all_MS[1] # Intensidades

In [ ]:
DataIn=Data[0:20480]
DataIn.columns=range(DataIn.shape[1])
DataMz=Datamz[0:20480]
DataMz.columns=range(DataMz.shape[1])

In [ ]:
#previsualización de las señales
plt.plot(DataIn);

In [ ]:
#gráfica de todas las señales
fig, ax = plt.subplots(figsize=(9,6))

plt.rcParams["font.family"] = 'Times New Roman'
ax.plot(DataIn, lw=2)
ax.set_xlim(0, 20480)
ax.set_ylim(0, 250000)
ax.set_xlabel('Number of data',  fontsize=16)
ax.set_ylabel('Intensity (arb. units)', fontsize=16)
fig.savefig("todosesp.png", dpi=200)
plt.show()

In [ ]:
EtPlanct = pd.read_excel('C:/Users/Usuario/OneDrive - UNIVERSIDAD INDUSTRIAL DE SANTANDER/Documents/Analisis_Datos_Quimica-main/Datos plan B/29213-43300/Extracción 1/Etiquetas.xlsx', sheet_name=0, header=None)

Y=EtPlanct[[2]]
Y1=pd.DataFrame(Y.set_axis(axis=1, labels=['Estado']))

In [ ]:

scaler=StandardScaler()
scaler.fit(DataIn) # calculo la media para poder hacer la transformacion
x_scaled=scaler.transform(DataIn)# Ahora si, escalo los datos y los normalizo

pca=PCA(n_components=10) # Otra opción es hacer pca hasta obtener un mínimo explicado ej.: pca=PCA(.85)
pca1=pca.fit(x_scaled.T) # obtener los componentes principales
datos_pca=pca.transform(x_scaled.T) # convertimos nuestros datos con las nuevas dimensiones de PCA, scores



print("shape of datos_pca", datos_pca.shape)
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:10]))

datos_pca1=pd.DataFrame(datos_pca) # Convierte los datos pca en un DataFrame
datos_pca1=pd.concat([Y1, datos_pca1] ,axis=1) # Se agrega la columna del tipo de muestra

In [ ]:
scores_df = pd.DataFrame(datos_pca, columns = [f"PC{i+1}" for i in range(datos_pca.shape[1])])
scores_df.head()

In [ ]:
from bokeh.io import output_notebook, show, output_file, export_png
from bokeh.plotting import figure, show

output_notebook()
#colormap = {'LB3D': 'red', 'LB7D': 'green', 'LP3D': 'blue', 'LP7D': 'black'}
colormap = {'Bio': 'blue', 'Pla': 'red'}
colors = [colormap[x] for x in Y1['Estado']]

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# Draw a chart
p = figure(title='', width=1600, height=1200,
           x_axis_label='PCA 1', y_axis_label='PCA 2', toolbar_location="above", tools=TOOLS)
p.grid.grid_line_alpha=0.3

p.circle(datos_pca1[0], datos_pca1[1],  color=colors, size=40, alpha=0.7)
#p.ygrid.band_fill_color = "olive"
#p.ygrid.band_fill_alpha = 0.1
#p.legend.location = "top_right"
p.grid.visible = True
p.title.align = "center"
p.title.text_font_size = "20px"
p.axis.axis_label_text_font_size = "40px"
p.axis.major_label_text_font_size = "30px"
#p.legend.location = "top_center"
export_png(p, filename="PCA Ext-1.png")

show(p)

In [ ]:
plt.figure(figsize=(10, 6))
bars = sns.barplot(x=np.arange(1, len(expl) + 1), y=expl, palette="summer")

plt.xlabel("Componentes principales",fontweight='bold',fontsize=12)
plt.ylabel("Varianza explicada",fontweight='bold',fontsize=12)
plt.savefig('Varianza Ext-1', dpi= 1200)

plt.show()